In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from tqdm.auto import tqdm, trange
from statsmodels.tsa.arima.model import ARIMAResults, ARIMA
import plotly.graph_objects as go

import arma
from arma import predict, pretrain, update, trial, train_model
from data import read_erdc_file, find_sequences, plot_seqs, probabilistic_distance, evaluate

pd.options.plotting.backend = "matplotlib"
READ_CSV=False
INTERVAL = pd.Timedelta(minutes=15)

## Real Data extraction/pre-processing

In [ ]:
df = read_erdc_file(processed=True)
df.describe()

In [ ]:
df_all = df # original dataframe with all data
# Testing data
dft = df_all.loc['2022-06-28':'2022-07-07']
# Training data
df = df_all.loc['2022-03-10':'2022-06-27']

In [ ]:
# separate into contiguous sequences
seqs = find_sequences(df, INTERVAL)
seqsog = find_sequences(df_all, INTERVAL)
print(len(seqs), len(seqsog))

In [ ]:
plot_seqs(seqsog)
plt.axvline(seqs[0].index[0], color='r')
plt.axvline(seqs[-1].index[-1], color='r')

## Timeseries Analysis

In [ ]:
# Stationarity
from statsmodels.tsa.stattools import kpss

# Null hypothesis: process is stationary

def kpss_test(timeseries):
    # print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    return kpss_output

tests = [kpss_test(s.Speed) for s in seqs]
pd.concat(tests, axis=1)

In [ ]:
# Correlation
corrs = df.rolling(1).mean().corr()
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corrs.columns,
        y = corrs.index,
        z = corrs,
        text=corrs.values,
        texttemplate='%{text:.2f}'
    )
)
fig.update_layout(height=300, width=300)
fig.show()

In [ ]:
# Filtering
from statsmodels.tsa.filters import hp_filter, bk_filter
seq = seqs[0]
plt.plot(seq.Speed)
cycle = bk_filter.bkfilter(seq.Speed, low=1e1, high=1e2, K=10)
plt.plot(seq.Speed.mean()+cycle)

In [ ]:
# Autocorrelation, used to select order of Moving Average (MA) in ARIMA models
plt.figure(figsize=(8,4))
ax = plt.subplot(1,1,1)
for s in seqsog:
    sm.graphics.tsa.plot_acf(s.Speed, lags=min(100, len(s)-1), ax=ax)
plt.xlabel(f'Time interval {s.index.freq}')
plt.tight_layout()

In [ ]:
# Partial Autocorrelation, used to select order of Autoregression (AR) in ARIMAX models
plt.figure(figsize=(8,4))
ax = plt.subplot(1,1,1)
for s in seqsog:
    sm.graphics.tsa.plot_pacf(s.Speed, lags=min(100, len(s)//2-1), ax=ax)
plt.xlabel(f'Time interval {s.index.freq}')
plt.tight_layout()

## ARIMA models

### Analysis

In [ ]:
model = sm.tsa.ARIMA(seqs[0].Speed.iloc[:-100], order=(1,0,20))
result = model.fit()
actual = seqs[0].Speed.iloc[-100:]
_, predictions = predict(result, actual, None, INTERVAL)


In [ ]:
# Error measure scaling with lag
df = pd.DataFrame({'lag':[], 'RMSE':[], 'Wasserstein':[], 'Jensen-Shannon':[]})
df.set_index(['lag'], inplace=True)
df

arr = np.random.randn(10000)
for lag in range(1,20):
    a = arr[lag:]
    b = arr[:len(arr)-lag]
    df.loc[lag] = [np.sqrt(np.mean(a-b)**2),
                   probabilistic_distance(a, b, measure='wasserstein'),
                   probabilistic_distance(a, b, measure='jensenshannon')]
df.plot()

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(predictions, label='1-Step')
# plt.plot(predictions_recursive, label='Recursive')
plt.plot(seqs[0].Speed.iloc[-100:], label='Actual')
plt.plot(predictions - actual, label='Residual', ls=':')
# plt.title('RMSE=%.2f' % sm.tools.eval_measures.rmse(predictions, actual))
plt.title('Jensen Shannon distance=%.2f' % probabilistic_distance(predictions, actual, measure='jensenshannon'))
# plt.title('RMSE=%.2f' % np.sqrt(np.mean((predictions - actual)**2)))
plt.legend()

In [ ]:
plt.figure(figsize=(12,4))
sim = result.simulate(int((seqs[1].index[-1] - seqs[0].index[0]) / seqs[0].index.freq))
seqs[0].Speed.plot(label='Sample 1')
seqs[1].Speed.plot(label='Sample 2')
sim.plot(label='Simulated')
plt.legend()

In [ ]:
_, bins = np.histogram(sim)
seqs[0].Speed.hist(bins=bins, density=True, alpha=0.3, label='Sample 1')
seqs[1].Speed.hist(bins=bins, density=True, alpha=0.3, label='Sample 2')
sim.hist(bins=bins, density=True, alpha=0.3, label='Simulated')
plt.legend()

In [ ]:
s0 = seqs[0].Speed
s1 = seqs[1].Speed
# bsim, lmbdasim = boxcox(sim)
b0, lmbda0 = boxcox(s0)
b1, lmbda1 = boxcox(s1)

fig, ax = plt.subplots(1,2, figsize=(12,4))

ax[0].hist(b0, bins=10, density=True, alpha=0.3, label='Sample 1')
ax[0].hist(b1, bins=10, density=True, alpha=0.3, label='Sample 2')
ax[0].hist(sim, bins=10, density=True, alpha=0.3, label='Simulated')
ax[0].set_title('Box-Cox transform')
ax[0].legend()

_, bins = np.histogram(sim)
ax[1].hist(inv_boxcox(b0, lmbda0), bins=bins, density=True, alpha=0.3, label='Sample 1')
ax[1].hist(inv_boxcox(b1, lmbda1), bins=bins, density=True, alpha=0.3, label='Sample 2')
ax[1].set_title('Inverse Box-Cox transform')
ax[1].legend()


In [ ]:
s0 = pd.Series(b0, index=seqs[0].index)
s1 = pd.Series(b1, index=seqs[1].index)

In [ ]:
model = sm.tsa.ARIMA(s0.iloc[:-100], order=(1,0,20))
result = model.fit()
actual = s0.iloc[-100:]
_, predictions = predict(result, actual, None, INTERVAL)


In [ ]:
plt.figure(figsize=(12,4))
sim = result.simulate(int((seqs[1].index[-1] - seqs[0].index[0]) / seqs[0].index.freq))
# sim_inv = in
s0.plot(label='Sample 1')
s1.plot(label='Sample 2')
sim.plot(label='Simulated')
plt.legend()

In [ ]:
_, bins = np.histogram(sim)
s0.hist(bins=bins, density=True, alpha=0.3, label='Sample 1')
s1.hist(bins=bins, density=True, alpha=0.3, label='Sample 2')
sim.hist(bins=bins, density=True, alpha=0.3, label='Simulated')
plt.legend()

In [ ]:
sim_inv = inv_boxcox(sim, lmbda0)
sim_inv = pd.Series(sim_inv, index=sim.index)
plt.figure(figsize=(12,4))
seqs[0].Speed.plot(label='Sample 1')
seqs[1].Speed.plot(label='Sample 2')
sim_inv.plot(label='Simulated')
plt.legend()

In [ ]:
_, bins = np.histogram(sim_inv)
seqs[0].Speed.hist(bins=bins, density=True, alpha=0.3, label='Sample 1')
seqs[1].Speed.hist(bins=bins, density=True, alpha=0.3, label='Sample 2')
sim_inv.hist(bins=bins, density=True, alpha=0.3, label='Simulated')
plt.legend()

In [ ]:
pred_inv = inv_boxcox(predictions, lmbda0)
pred_inv = pd.Series(pred_inv, index=predictions.index)
plt.figure(figsize=(8,4))
plt.plot(pred_inv, label='1-Step')
# plt.plot(predictions_recursive, label='Recursive')
plt.plot(seqs[0].Speed.iloc[-100:], label='Actual')
plt.plot(pred_inv - actual, label='Residual', ls=':')
# plt.title('RMSE=%.2f' % sm.tools.eval_measures.rmse(predictions, seqs[0].Speed.iloc[-100:]))
plt.title('Jensen Shannon distance=%.2f' % probabilistic_distance(pred_inv, seqs[0].Speed.iloc[-100:], measure='jensenshannon'))
# plt.title('RMSE=%.2f' % np.sqrt(np.mean((pred_inv - seqs[0].Speed.iloc[-100:])**2)))
plt.legend()

### Evaluation on real data

In [ ]:
# real data
dftrain = pd.DataFrame(seqs[0].Speed)
dftest = pd.DataFrame(seqs[1].Speed)
# Persistence
evaluate(dftest.Speed.iloc[:-1], dftest.Speed.iloc[1:], 'Persistence')

In [ ]:
model_arma = arma.train(ARIMA, dftrain, exog=None, interval=INTERVAL, model_kwargs={'order':(1,0,20)})

In [ ]:
model_arma = model_arma.apply(dftest.Speed)
p = model_arma.predict(start=dftest.index[0], end=dftest.index[-1], exog=None, dynamic=False)
pp = model_arma.get_prediction(start=dftest.index[0], end=dftest.index[-1], exog=None, dynamic=False) # w/ confidence bounds

In [ ]:
plt.figure(figsize=(16,4))
ci = pp.conf_int()
pp.predicted_mean.plot(ax=plt.gca(), lw='0.5', label='predicted')
plt.fill_between(ci.index, ci['lower Speed'], ci['upper Speed'], alpha=0.5, color='r', label='95% ci')
dftest.Speed.plot(lw=0.5, ls='-.', color='k', ax=plt.gca(), label='actual')
plt.legend()

In [ ]:
inside = (ci['lower Speed'] < dftest.Speed) & (dftest.Speed < ci['upper Speed'])
inside.sum() / len(inside)

In [ ]:
evaluate(p, dftest.Speed, 'ARMA')

## LSTM

In [ ]:
import torch
# from torch.nn.utils.rnn import pack_sequence, unpack_sequence
from lstm import Model, train_lstm

model_lstm = Model()
loss = train_lstm(model_lstm, lr=0.001, data=dftrain.Speed, epochs=300)
plt.plot(loss)

In [ ]:
p = model_lstm(torch.from_numpy(dftest.Speed.values).float()).detach().numpy().squeeze()

evaluate(p, dftest.Speed, 'LSTM')

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(dftest.Speed.values, lw=0.2)
plt.plot(p, lw=0.3, label='predicted')
plt.legend()

# Synthetic data Model evaluation

In [ ]:
df = pd.read_csv('./synthetic.csv', index_col=0, parse_dates=True)
df.index.freq = INTERVAL

n = int(len(df) * 1)
train_size = int(0.5 * n)
test_size = int(0.5 * n)
dftrain = df.iloc[:train_size]
dftest = df.iloc[-test_size:]

# Persistence
evaluate(dftest.Speed.iloc[:-1], dftest.Speed.iloc[1:], 'Persistence')

### ARMA

In [ ]:
model_arma = pretrain(ARIMA, dftrain.Speed, exog=None, order=(1,0,20))
model_arma.remove_data() # lower memory usage

In [ ]:
model_arma = model_arma.apply(dftest.Speed)
p = model_arma.predict(start=dftest.index[0], end=dftest.index[-1], exog=None, dynamic=False)
pp = model_arma.get_prediction(start=dftest.index[0], end=dftest.index[-1], exog=None, dynamic=False) # w/ confidence bounds

In [ ]:
plt.figure(figsize=(16,4))
ci = pp.conf_int()
pp.predicted_mean.plot(ax=plt.gca(), lw='0.5', label='predicted')
plt.fill_between(ci.index, ci['lower Speed'], ci['upper Speed'], alpha=0.5, color='r', label='95% ci')
dftest.Speed.plot(lw=0.5, ls='-.', color='k', ax=plt.gca(), label='actual')
plt.legend()

In [ ]:
inside = (ci['lower Speed'] < dftest.Speed) & (dftest.Speed < ci['upper Speed'])
inside.sum() / len(inside)

In [ ]:
evaluate(p, dftest.Speed, 'ARMA')

### LSTM

In [ ]:
import torch
# from torch.nn.utils.rnn import pack_sequence, unpack_sequence
from lstm import Model, train_lstm

model_lstm = Model()
loss = train_lstm(model_lstm, lr=0.001, data=dftrain.Speed, epochs=100)
plt.plot(loss)

In [ ]:
model_lstm = model
p = model_lstm(torch.from_numpy(dftest.Speed.values).float()).detach().numpy().squeeze()

evaluate(p, dftest.Speed, 'LSTM')

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(dftest.Speed.values, lw=0.2)
plt.plot(p, lw=0.3, label='predicted')
plt.legend()

## Transfer learning

In [ ]:
df = pd.read_csv('./synthetic.csv', index_col=0, parse_dates=True)
df.index.freq = INTERVAL

n = int(len(df) * 1)
train_size = int(0.5 * n)
test_size = int(0.5 * n)
dftrain = df.iloc[:train_size]

dftest = pd.DataFrame(seqs[1].Speed)

### ARMA

In [ ]:
model_arma = pretrain(ARIMA, dftrain.Speed, exog=None, order=(1,0,20))
model_arma.remove_data() # lower memory usage

In [ ]:
model_arma = model_arma.apply(dftest.Speed)
p = model_arma.predict(start=dftest.index[0], end=dftest.index[-1], exog=None, dynamic=False)
pp = model_arma.get_prediction(start=dftest.index[0], end=dftest.index[-1], exog=None, dynamic=False) # w/ confidence bounds

In [ ]:
inside = (ci['lower Speed'] < dftest.Speed) & (dftest.Speed < ci['upper Speed'])
inside.sum() / len(inside)

In [ ]:
evaluate(p, dftest.Speed, 'ARMA')

In [ ]:
plt.figure(figsize=(16,4))
ci = pp.conf_int()
pp.predicted_mean.plot(ax=plt.gca(), lw='0.5', label='predicted')
plt.fill_between(ci.index, ci['lower Speed'], ci['upper Speed'], alpha=0.5, color='r', label='95% ci')
dftest.Speed.plot(lw=0.5, ls='-.', color='k', ax=plt.gca(), label='actual')
plt.legend()

### LSTM

In [ ]:
import torch
# from torch.nn.utils.rnn import pack_sequence, unpack_sequence
from lstm import Model, train_lstm

model_lstm = Model()
loss = train_lstm(model_lstm, lr=0.001, data=dftrain.Speed, epochs=100)
plt.plot(loss)

p = model_lstm(torch.from_numpy(dftest.Speed.values).float()).detach().numpy().squeeze()

evaluate(p, dftest.Speed, 'LSTM')

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(dftest.Speed.values, lw=0.2)
plt.plot(p, lw=0.3, label='predicted')
plt.legend()